In [2]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [3]:
# Using HuggingFace's QuestionAnsweringModel (inbuilt)

import tensorflow as tf

# Refer: https://huggingface.co/transformers/model_doc/bert.html

from transformers import BertTokenizer, TFBertForQuestionAnswering 

modelName = 'bert-large-uncased-whole-word-masking-finetuned-squad' # https://huggingface.co/transformers/pretrained_models.html
 
tokenizer = BertTokenizer.from_pretrained(modelName)
model = TFBertForQuestionAnswering.from_pretrained(modelName)

print(model)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

2022-02-06 14:23:51.099530: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-06 14:23:51.100522: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [4]:
# Text+Q&A from SQuAD Test data: https://rajpurkar.github.io/SQuAD-explorer/explore/v2.0/dev/Amazon_rainforest.html?model=nlnet%20(single%20model)%20(Microsoft%20Research%20Asia)&version=v2.0
text = r"""The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species."""
question =r"""What percentage does the Amazon represents in rainforests on the planet?"""

#question = r"""How many nations contain "Amazonas" in their names?"""


#input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
input_text =  question + " [SEP] " + text 
input_ids = tokenizer.encode(input_text)

print(len(input_ids))
print(tokenizer.decode(input_ids))

input = tf.constant(input_ids)[None, :]  # Batch size 1 


250
[CLS] what percentage does the amazon represents in rainforests on the planet? [SEP] the amazon rainforest ( portuguese : floresta amazonica or amazonia ; spanish : selva amazonica, amazonia or usually amazonia ; french : foret amazonienne ; dutch : amazoneregenwoud ), also known in english as amazonia or the amazon jungle, is a moist broadleaf forest that covers most of the amazon basin of south america. this basin encompasses 7, 000, 000 square kilometres ( 2, 700, 000 sq mi ), of which 5, 500, 000 square kilometres ( 2, 100, 000 sq mi ) are covered by the rainforest. this region includes territory belonging to nine nations. the majority of the forest is contained within brazil, with 60 % of the rainforest, followed by peru with 13 %, colombia with 10 %, and with minor amounts in venezuela, ecuador, bolivia, guyana, suriname and french guiana. states or departments in four nations contain " amazonas " in their names. the amazon represents over half of the planet's remaining rainf

In [5]:
# Source: Modified PyTorch code from https://www.kaggle.com/c/tensorflow2-question-answering/discussion/123434

#[SEP] =  102 (token_id)
# token_type_ids has 0 corresponding to the qeustion part of the input_text and 1 corresponding to the 
# text/part 

# here we don't want output corresponding to question that' why we keep value as 0 till index of [SEP] token. 
#We want outputs corresponding to para hence we keep value as 1 after [SEP] token
token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]

print(token_type_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [12]:
answer=model(input, token_type_ids = tf.convert_to_tensor([token_type_ids]))

print(type(answer))
print(len(answer))

# https://huggingface.co/transformers/model_doc/distilbert.html#tfdistilbertforquestionanswering

startScores, endScores = answer.to_tuple()
print(startScores.shape)
print(endScores.shape)
print(type(startScores))


<class 'transformers.modeling_tf_outputs.TFQuestionAnsweringModelOutput'>
2
(1, 250)
(1, 250)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [13]:
input_tokens = tokenizer.convert_ids_to_tokens(input_ids)

print(input_tokens)

['[CLS]', 'what', 'percentage', 'does', 'the', 'amazon', 'represents', 'in', 'rainforest', '##s', 'on', 'the', 'planet', '?', '[SEP]', 'the', 'amazon', 'rainforest', '(', 'portuguese', ':', 'flores', '##ta', 'amazon', '##ica', 'or', 'amazon', '##ia', ';', 'spanish', ':', 'se', '##lva', 'amazon', '##ica', ',', 'amazon', '##ia', 'or', 'usually', 'amazon', '##ia', ';', 'french', ':', 'fore', '##t', 'amazon', '##ien', '##ne', ';', 'dutch', ':', 'amazon', '##ere', '##gen', '##wo', '##ud', ')', ',', 'also', 'known', 'in', 'english', 'as', 'amazon', '##ia', 'or', 'the', 'amazon', 'jungle', ',', 'is', 'a', 'moist', 'broad', '##leaf', 'forest', 'that', 'covers', 'most', 'of', 'the', 'amazon', 'basin', 'of', 'south', 'america', '.', 'this', 'basin', 'encompasses', '7', ',', '000', ',', '000', 'square', 'kilometres', '(', '2', ',', '700', ',', '000', 'sq', 'mi', ')', ',', 'of', 'which', '5', ',', '500', ',', '000', 'square', 'kilometres', '(', '2', ',', '100', ',', '000', 'sq', 'mi', ')', 'are', 

In [14]:
print(startScores)

tf.Tensor(
[[-6.047486   -4.656021   -7.571983   -7.81299    -7.0066047  -7.89475
  -8.126448   -7.5400686  -8.892704   -9.418995   -8.942823   -8.615164
  -9.496939   -9.665028   -6.0474596  -6.15179    -3.5666277  -7.074653
  -8.149385   -6.5617304  -8.559914   -7.1163764  -8.553457   -7.5595956
  -8.405012   -8.682978   -6.790607   -8.405868   -8.498199   -6.235168
  -8.774739   -7.628982   -8.575292   -7.887274   -8.5714855  -8.881729
  -7.3598723  -8.744013   -8.553474   -6.743693   -7.268454   -8.534403
  -8.565097   -6.486239   -8.576063   -6.986379   -8.542222   -7.4793887
  -8.858678   -8.480017   -8.771891   -6.917669   -8.668295   -7.1957245
  -8.835161   -8.785323   -8.765894   -8.421244   -7.567758   -7.9901147
  -7.5482965  -7.8218055  -8.1223955  -7.2564707  -8.40498    -6.5453877
  -8.211744   -8.5522     -7.2628655  -6.084313   -7.759269   -8.03882
  -7.1379924  -7.301931   -7.178515   -7.6427255  -8.358979   -7.0667424
  -8.150647   -7.197264   -6.813742   -8.397957  

In [15]:
print(tf.math.argmax(startScores[0],0))

tf.Tensor(207, shape=(), dtype=int64)


In [16]:
print((tf.math.argmax(startScores[0],0)).numpy())

207


In [17]:
startIdx = tf.math.argmax(startScores[0],0).numpy()
endIdx = tf.math.argmax(endScores[0],0).numpy()+1
print(startIdx,endIdx)

207 209


In [18]:
print(" ".join(input_tokens[startIdx:endIdx]))

over half
